In [1]:
#!/usr/bin/python

import numpy as np
from astropy.io import fits as fits
from matplotlib import pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
from astropy.wcs import WCS

from scipy import signal

In [2]:
s_SSP_id, s_age_SSP, s_met, s_logML = np.loadtxt('SSP_logML.txt', unpack='True')
s_SSP_Mstar_sorted = np.loadtxt('SSP_Mstar_sorted.txt', usecols=(3,))
s_ML_SSP = 10**s_logML
s_age_SSP_sorted = np.argsort( s_age_SSP )
s_age = np.unique(s_age_SSP)
#s_tau = ( np.r_[ -s_age[0], s_age ] + np.r_[ s_age, 2*s_age[-1]-s_age[-2] ] ) / 2.
s_tau = np.sqrt( np.r_[ 0, s_age ] * np.r_[ s_age, 2*s_age[-1]-s_age[-2] ] ) * 1e9 # geometric mean between SSP ages
s_tau_now = np.zeros(len(s_tau))
s_tau_now[0] = 1e6
s_today = s_tau[-1]
s_N_ages = s_age.size
s_N_met  = np.unique(s_met).size
#print "Pipe3D: ({} ages) x ({} metallicities) = {} SSPs".format( s_N_ages, s_N_met, s_age_SSP.size )

# Cosmic time (years)
AgeUnivers = s_tau[len(s_tau)-1]
t = AgeUnivers - s_tau

# Correction for numerical problems
t = t[:len(t)-1]

# Linear Cosmic Time
dt= (t[0]-t[1])
time = t[0] - np.arange(t[0]/dt) *dt

# Recent time to calculate de mean SN CC Rate (Million of years)
recent_time = 10
n = np.int(np.round(recent_time / (dt/1e6)))


In [3]:
class Pipe3D_galaxy:
    """
    Class that reads Pipe3D results.
    """
#------------------------------------------------------------------------------

    #--------------------------------------------------------------------------
    def __init__(self, name):
        """
        The name of the galaxy (root of the Pipe3D output files) must be passed to the constructor
        """
        self.name = name
        self.time = time
        #self.dtd = dtd
        
        hdu = fits.open( '../Data/Amusing/'+name+'.SSP.cube.fits' )
        self.segmentation_map    = hdu[0].data[ 1]
        self.flux_map            = hdu[0].data[ 3] * 1 / (3.826e33 * 1.05e-37) # in Lsun/parsec**2
        self.surface_density_map = hdu[0].data[18] # not dust corrected
        [ self.Ny, self.Nx ] = self.segmentation_map.shape
        self.zones = np.unique( self.segmentation_map )
        self.Nzones = len(self.zones)
        hdu.close()
        
        hdu = fits.open( '../Data/Amusing/'+name+'.SFH.cube.fits' )
        self.light_fraction = hdu[0].data[:s_age_SSP.size]
        hdu.close()


    #--------------------------------------------------------------------------
    def cumulative_surface_density_spaxel(self, i, j, stars_die ):
        """
        Compute the mass history Sigma(tau) for spaxel (i,j),
        where tau is the stellar age,
        and Sigma(tau) is the mass surface density, in Msun/pc**2, of stars older than tau.
        The boolean parameter stars_die tells whether the function should return
        the total mass of stars formed (stars_die = False)
        or the mass of stars that are alive at the present time (stars_die = True).
        """
        light_fraction = self.light_fraction[:,i,j]
        mass_fraction = light_fraction * s_ML_SSP
        cumulative_mass = np.empty( s_N_ages )
        mass_so_far = 0.
        age_index = s_N_ages
        last_age = s_age_SSP[ s_age_SSP_sorted[-1] ]
          
        for SSP_index, Mstar in zip( s_age_SSP_sorted[::-1], s_SSP_Mstar_sorted[::-1] ):
            if s_age_SSP[SSP_index] != last_age:
                age_index = age_index-1
                cumulative_mass[age_index] = mass_so_far
                last_age = s_age_SSP[SSP_index]
                #print "index {}: M({})={}".format( age_index, last_age, cumulative_mass )
            if( stars_die ): mass_so_far += mass_fraction[SSP_index] * Mstar
            else:            mass_so_far += mass_fraction[SSP_index]
            #print i,j, s_age_SSP[SSP_index], Mstar
        cumulative_mass[0] = mass_so_far

        return self.flux_map[i,j] * np.r_[ cumulative_mass, 0. ] # En Masas solares por parsec cuadrado.


    #--------------------------------------------------------------------------
    def painting_galaxy_maps(self, stars_die, alfa, delta, sigma ):
        """
        stars_die = True returns only "still-burning" stars.
        stars_die = False returns all mass in stars ever formed.
        alfa = minus power index of the DTD.
        delta = time in My of no SN in the DTD.
        sigma = time in My of core collapse distribution.
        
        """
        self.rateIa = np.zeros(self.Nzones)
        
        # Delay Time Distribution (SN per Year per Solar Mass)
        delta = int(delta*1e6/dt)
        dtd = 1e-4 * time**-alfa
        dtd[len(time)-delta:] = 0
        
        for zone in np.arange(self.Nzones):
            i_zone=0
            j_zone=0
            
            for i in range(self.Ny):
                if zone == int(self.segmentation_map[i_zone,j_zone]):
                    break
                
                for j in range(self.Nx):
                    if zone == int(self.segmentation_map[i,j]):
                        i_zone = i
                        j_zone = j
                        break
                
            # Reading the data.
            mass_density = self.cumulative_surface_density_spaxel(i_zone, j_zone, stars_die)
            # Correction for numerical problems
            mass_density = mass_density[:len(mass_density)-1]
            
            # Linear Cumulative Mass (Solar Mass)
            cumulative_mass = np.interp(-time, -t, mass_density)
             
            # Star formation rate (Solar Mass per years)
            sfr = -np.gradient(cumulative_mass) / dt
            base = len(sfr)
            
            # Sigma of the gaussian in Millions of years
            temporal_smooth = 100
            gaus = signal.gaussian(len(time),temporal_smooth*1e6/dt)
            extra = len(gaus[gaus>np.max(gaus)*1e-3])
            
            sfr = np.convolve(sfr,gaus[gaus>np.max(gaus)*1e-3])/np.sum(gaus[gaus>np.max(gaus)*1e-3])
            
            # Resize
            sfr = sfr[extra/2:base+extra/2]
            
            # SN Ia rate (SN per Year)
            snr = dt * np.convolve(sfr,dtd,'valid')
            
            # Final results.
            
            # Rate of SN Ia (SN per year)
            self.rateIa[zone] = snr[0]
            
        # Galaxy maps
        self.SNIa_rate_map = np.interp(self.segmentation_map,self.zones,self.rateIa)
        
        mask = self.flux_map
        mask[mask<0] = 0
        mask[mask>0] = 1
        mask[mask==0] = 0
        
        self.SNIa_rate_map = self.SNIa_rate_map * mask
                
    #--------------------------------------------------------------------------
#------------------------------------------------------------------------------